## Nombre y apellido

...

## Importamos librerías

In [ ]:
import math
import random

import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

#from deap import algorithms
from deap import base  # Estructura que permite agrupar todos los componentes de nuestro algoritmo en una misma bolsa
from deap import creator  # Permite crear los componentes de nuestro algoritmo
from deap import tools  # Contiene funciones precargadas

from joblib import Parallel, delayed

<div style='background:#A2FFF2; padding:10px;'>
<h1>Enunciado</h1>
Se desea encontrar el mínimo de la función que se presenta en la celda siguiente, dentro del intervalo [$-500$,$500$].
    
Proponga un algoritmo genético capaz de resolver esta tarea.
</div>

In [ ]:
def F(x,y):
    '''
    Función a optimizar.
    '''
    z = 418.9829 * 2 - x * np.sin( np.sqrt( np.abs( x )))- y * np.sin( np.sqrt(np.abs(y)))

    return z

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

V = np.arange(-500, 500.1, 0.1)

X,Y  = np.meshgrid(V, V)

Z = F(X,Y)

# GRAFICAMOS SUPERFICIE
cs = ax.contourf(X,Y,Z, cmap='cool')

ax.grid(True)
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)

fig.colorbar(cs)
plt.show()

---

## Funciones auxiliares

In [ ]:
#=================================
def bin(p=0.5):
    '''
    Esta función genera un bit al azar.
    '''
    if random.random() < p:
        return 1
    else:
        return 0
#=================================


#=================================
def mutation(ind, p):
    '''
    Esta función recorre el cromosoma y evalúa, para cada gen,
    si debe aplicar el operador de mutación.
    '''
    
    return [abs(i-1) if random.random() < p else i for i in ind]
#=================================

**CONSIGNA 1**: Implemente una función para realizar el mapeo del genotipo al fenotipo.

In [ ]:
def bin2dec(ind, low, high):
    '''
    Esta función permite convertir un número binario
    (lista de "0" y "1") en un valor decimal, dados
    los límites inferior y superior tomados para la
    conversión.
    low: Límite inferior del rango a barrer
    high: Límite superior del rango a barrer
    '''

    ...

    return x, y

**CONSIGNA 2**: Implemente la función de fitness para guiar la optimización.

In [ ]:
def fitness(ind, low, high):
    '''
    Función de aptitud empleada por nuestro algoritmo.
    '''
    
    ...
    
    return z

## Inicializamos parámetros del experimento

**CONSIGNA 3**: Proponga valores para los siguientes parámetros:
- `IND_SIZE`   # Cantidad total de genes en el cromosoma
- `LB`         # Límite inferior del rango a evaluar
- `UB`         # Límite superior del rango a evaluar
- `POP_SIZE`   # Cantidad de individuos en la población
- `PM`         # Probabilidad de mutación
- `N_PARENTS`  # Número de padres para la cruza
- `PX`         # Probabilidad de cruza
- `GMAX`       # Cantidad máxima de generaciones

Justifique su elección.

In [ ]:
IND_SIZE = ...  # Cantidad total de genes en el cromosoma
LB = ...        # Límite inferior del rango a evaluar
UB = ...        # Límite superior del rango a evaluar

POP_SIZE = ...  # Cantidad de individuos en la población
PM = ...        # Probabilidad de mutación [aproximadamente 1 gen por cromosoma]
N_PARENTS = ... # Número de padres seleccionados para generar la descendencia
PX = ...        # Probabilidad de cruza
GMAX = ...      # Cantidad máxima de generaciones que se ejecutará el algoritmo

## Creamos los componentes del algoritmo

In [ ]:
# CREAMOS LA FUNCION DE FITNESS
# Esta función tiene "1 OBJETIVO" a "MINIMIZAR"
creator.create("Fitness",  # Nombre con el que se registra el componente
               base.Fitness,  # Clase de la que hereda
               weights=(...,))

#---------------------

# CREAMOS EL CONSTRUCTOR DE INDIVIDUOS
creator.create("Individual", # Nombre con el que se registra el componente
               list,  # Clase de la que hereda [Tipo de contenedor en este caso]
               fitness=creator.Fitness)  # Asignamos un método para evaluar el fitness del individuo

## REGISTRAMOS COMPONENTES

In [ ]:
toolbox = base.Toolbox()

#---------------------

# DEFINIMOS COMO CONSTRUIR UN GEN
toolbox.register("attribute",  # Nombre con el que se registra el componente
                 bin,
                 p=0.5)  # Función asociada a ese componente

#---------------------

# DEFINIMOS COMO CONSTRUIR UN INDIVIDUO/CROMOSOMA
toolbox.register("individual",  # Nombre con el que se registra el componente
                 tools.initRepeat,  # Método usado para construir el cromosoma
                 creator.Individual,  # ...
                 toolbox.attribute,  # Función para construir cada gen
                 n=IND_SIZE)  # Número de genes del cromosoma/individuo (se repetirá la función construir gen)

#---------------------

# DEFINIMOS COMO CONSTRUIR LA POBLACION
toolbox.register("population",  # Nombre con el que se registra el componente
                 tools.initRepeat,  # Método usado para construir el cromosoma
                 list,
                 toolbox.individual)

#---------------------

# DEFINIMOS COMO REALIZAR LA CRUZA
toolbox.register("mate",  # Nombre con el que se registra el componente
                 tools.cxTwoPoint)  # 

#---------------------

# DEFINIMOS COMO REALIZAR LA MUTACION
toolbox.register("mutate",  # Nombre con el que se registra el componente
                 mutation,  # Método de mutación (definido como función más arriba)
                 p=PM)  # Parámetro que usa la mutación

#---------------------

# DEFINIMOS COMO REALIZAR LA SELECCION DE INDIVIDUOS
toolbox.register("select",  # Nombre con el que se registra el componente
                 tools.selTournament,  # Método usado para selección [selRoulette | selTournament | ...]
                 tournsize=5)  # Parámetro que usa el torneo

## Definimos las estadísticas a calcular

In [ ]:
# EXTRAEMOS EL FITNESS DE TODOS LOS INDIVIDUOS
stats_fit = tools.Statistics(key=lambda ind: ind.fitness.values)

# EXTRAEMOS EL TAMAÑO DE TODOS LOS INDIVIDUOS
stats_size = tools.Statistics(key=len)

# EXTRAEMOS EL TAMAÑO DE TODOS LOS INDIVIDUOS
stats_active_genes = tools.Statistics(key=lambda ind: np.sum(ind))

mstats = tools.MultiStatistics(fitness=stats_fit,
                               size=stats_size,
                               genes=stats_active_genes)

mstats.register("avg", np.mean)
mstats.register("std", np.std)
mstats.register("min", np.min)
mstats.register("max", np.max)

# INICIALIZAMOS UN LOGGER
logbook = tools.Logbook()

In [ ]:
#================================================
# INICIALIZAMOS LA POBLACIÓN
#================================================
pop = toolbox.population(n=POP_SIZE)  # Inicializamos una población
#================================================


#================================================
# EVALUAMOS EL FITNESS DE LA POBLACION
#======================================
fitnesses = Parallel(n_jobs=6, backend='multiprocessing')(delayed(fitness)(ind, LB, UB) for ind in pop)
#================================================


#================================================
# ASIGNAMOS A CADA INDIVIDUO SU FITNESS
#========================================
for ind,fit in zip(pop, fitnesses):
    ind.fitness.values = (fit,)  # Guardamos el fitness para cada individuo (en el individuo)
#================================================

records = mstats.compile(pop)
logbook.record(gen=0, **records)

**CONSIGNA 4**: Proponga el ciclo de pasos a seguir por el algoritmo para realizar la evoluión.

In [ ]:
#################################################################################
# COMENZAMOS LA EVOLUCION
################################

for g in range(1,GMAX):

    ...
    
    #================================================
    # CALCULAMOS ESTADÏSTICAS
    #============================
    records = mstats.compile(pop)
    logbook.record(gen=g, **records)

    if (g%10 == 0):
        print('='*79)
        print(f'GENERATION: {g}')
        print(f'ELITE -- Fitness: {elite.fitness.values[0]:.4}')
        print('FITNES: ', records['fitness'])
    #================================================

## Graficamos la evolución del fitness

In [ ]:
f_avg = [f['avg'] for f in logbook.chapters['fitness']]  # Extraemos fitness promedio a lo largo de las épocas
f_max = [f['max'] for f in logbook.chapters['fitness']]  # Extraemos fitness máximo a lo largo de las épocas
f_min = [f['min'] for f in logbook.chapters['fitness']]  # Extraemos fitness mínimo (elite) a lo largo de las épocas

fig, ax = plt.subplots(1, 1, figsize=(20,6)) 
ax.plot(range(GMAX), f_avg, '-r')
ax.plot(range(GMAX), f_max, '-g')
ax.plot(range(GMAX), f_min, '-b')
ax.set_xlabel('Generaciones', fontsize=16)
ax.set_ylabel('Fitness', fontsize=16)
ax.grid(True)

**CONSIGNA 5**: ¿Qué puede concluir de estas curvas de evolución?

...

**CONSIGNA 6**: ¿Cuáles son las coordenadas del mejor individuo?

In [ ]:
x_elite = ...
y_elite = ...

## Graficamos la solución

In [ ]:
#====================================================
# PLOT RESULTS
#===================
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

delta = 0.1
V = np.arange(LB, UB+delta, delta)

X,Y  = np.meshgrid(V, V)

Z = F(X,Y)

# GRAFICAMOS SUPERFICIE
cs = ax.contourf(X,Y,Z, cmap='cool')

ax.scatter(x_elit, y_elit, s=30, c='k')

ax.grid(True)
ax.set_xlabel('X', fontsize=16)
ax.set_ylabel('Y', fontsize=16)

fig.colorbar(cs)
plt.show()

**CONSIGNA 7**: ¿Cómo modificaría la función de mapeo de genotipo a fenotipo para que sea capaz de representar con diferente resolución los valores de `x` e `y`? Por ejemplo, `x` usando 8 bits e `y` usando 16 bits.

**CONSIGNA 8**: Ejecute el algoritmo 5 veces, empleando en cada caso 8 y 32 bits para representar cada variable. Construya una tabla con las coordenadas obtenidas en cada caso. ¿Se conserva la solución? Comente al respecto.